In [1]:
import os, sys, timeit

import numpy as np
import pandas as pd

from metrics import process

from tqdm import tqdm
from os.path import join
from kmeans_clustering import KMeansClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.mixture import GaussianMixture

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
pwd = os.getcwd()
data_dir = join(pwd, 'data/')
need_normalization = True
need_standardization = False
need_pca = False
filename = 'iris.csv'
datapath = join(data_dir, filename)
num_clusters = 5
num_iterations = 300
num_iter_exp = 20
verbose = False

In [3]:
# f = open('data/data.csv')
# lines = f.read()
# print(type(lines))
# new_lines = ''
# for line in lines:
#     new_lines += str(line.encode('utf-8').strip())
# print(len(new_lines))

In [4]:
df = pd.read_csv(
    datapath, 
    low_memory=False,
    encoding='utf-8',
#     sep=',',
    header=None,
)

In [5]:
print(len(df))

151


In [6]:
print(df.head())

              0            1             2            3        4
0  sepal.length  sepal.width  petal.length  petal.width  variety
1           5.1          3.5           1.4           .2   Setosa
2           4.9            3           1.4           .2   Setosa
3           4.7          3.2           1.3           .2   Setosa
4           4.6          3.1           1.5           .2   Setosa


In [7]:
print(df.dtypes)

0    object
1    object
2    object
3    object
4    object
dtype: object


In [8]:
columns = df.columns.tolist()
cols = columns
# cols = columns[1:]
# cols = ['Survived', 'Pclass', 'Age', 'SibSp', 'Fare']
cols = columns[:-1]
# cols = ['Passenger Count', 'Adjusted Passenger Count']
# cols = columns[1:]
# cols = columns[2:]
df = df[cols]
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.fillna(0)
df = df.replace([-np.inf, np.inf], np.nan)
df = df.dropna()
df = df.loc[:, (df != 0).any(axis=0)]
print(len(df))
print(df.dtypes)

151
0    float64
1    float64
2    float64
3    float64
dtype: object


In [9]:
df = df.sample(
    frac=1.0,
    random_state=1,
)
df = df.reset_index(drop=True)

In [10]:
if need_normalization is True:
    normalizer = StandardScaler()
    tmp = normalizer.fit_transform(df)
    df = pd.DataFrame(tmp, columns=df.columns)
elif need_standardization is True:
    scaler = MinMaxScaler()
    tmp = scaler.fit_transform(df)
    df = pd.DataFrame(tmp, columns=df.columns)

In [11]:
X = df
if need_pca is True:
    pca = PCA(
        n_components=len(cols),
        svd_solver='auto',
    )
    X = pca.fit_transform(X)
else:
    X = X.to_numpy()

In [12]:
for x in X:
    for a in x:
        if np.isinf(a):
            print(x)

In [13]:
data = X
print(data[:5])
print(data.shape)

[[-1.58493671 -0.07432758 -1.47939109 -1.42965201]
 [ 0.41646444 -0.2747466   0.3185007   0.1422712 ]
 [ 0.62713825 -0.2747466   0.3185007   0.1422712 ]
 [-0.11022007  2.73153862 -1.25465462 -1.03667121]
 [ 1.68050728 -0.47516561  1.32981483  0.9282328 ]]
(151, 4)


In [14]:
models = ['coc', 'kmeans++', 'variance', 'ostrovsky']
# models = ['kmeans++', 'kmeans++_corrected']

In [15]:
def plot_scatter(X, labels):
    fig, ax = plt.subplots()
    scatter = plt.scatter(
        X[:, 0],
        X[:, 1],
        c=labels,
    )
    handles, labels = scatter.legend_elements()
    legend = ax.legend(
        handles,
        labels,
        loc='upper right',
        title='Label',
    )
    ax.add_artist(legend)
    plt.title(model+' '+str(num_clusters)+ ' clusters')
    plt.xlabel('Component A')
    plt.ylabel('Component B')
    plt.show()

In [16]:
algorithms = {}
iterations = {}
inertias = {}
times = {}
inertias_min = {}
iters_min = {}
for model in models:
    iterations[model] = 0.0
    inertias[model] = 0.0
    times[model] = 0.0
    iters_min[model] = 0
    inertias_min[model] = 1e20
for i in tqdm(range(num_iter_exp)):
    for model in (models):
        algorithms[model] = KMeansClustering(
            n_clusters=num_clusters,
            max_iter=num_iterations,
            init=model,
            verbose=verbose,
        )
        start = timeit.default_timer()
        algorithm = algorithms[model].fit(data)
        centers = algorithm.cluster_centers_
        labels = algorithms[model].predict(data)
#         plot_scatter(X, labels)
        inertias[model] += algorithm.best_inertia_
        inertias_min[model] = min(inertias_min[model], algorithm.best_inertia_)
        iters_min[model] += algorithm.n_iters_
        end = timeit.default_timer()
        times[model] += (end-start)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:00<00:00,  3.05s/it]


In [17]:
for model in models:
    print(model+', '+str(num_clusters))
    print('Average Inertia -', round(inertias[model]/num_iter_exp, 2))
    print('Minimum Inertia - ', round(inertias_min[model], 2))
    print('Time -', round(times[model]/num_iter_exp, 2))
#     print('Average number of iterations', round(iters_min[model]/num_iter_exp,2))
    print('')

coc, 5
Average Inertia - 105.44
Minimum Inertia -  91.82
Time - 0.76

kmeans++, 5
Average Inertia - 107.65
Minimum Inertia -  91.82
Time - 0.76

variance, 5
Average Inertia - 99.53
Minimum Inertia -  91.82
Time - 0.77

ostrovsky, 5
Average Inertia - 101.5
Minimum Inertia -  91.82
Time - 0.76

